## Приступаем к обучению 2-ой ступени системы.

Пройдём 3 этапа:
- Анализ
- Обработка
- Обучение модели


# Анализ

In [8]:
import pandas as pd
import numpy as np
import re

users = pd.read_csv('./dataset/users.csv')
items = pd.read_csv('./dataset/items.csv')

estimates = pd.read_csv('./dataset/interactions.csv')


### Users

Взглянем сначала на таблицу юзеров.

In [9]:
users.sample(100)

,user_id,age,sex
3602,4006,18_24,0.0
16701,18663,18_24,0.0
99194,110735,45_54,0.0
36526,40753,18_24,0.0
52467,58551,45_54,1.0
...,...,...,...
38279,42718,55_64,0.0
80201,89525,45_54,0.0
23430,26119,35_44,0.0
137654,153740,18_24,0.0


Посчитаем пропуски.

In [10]:
l = len(users)

miss_age = users.age.isna().sum()
miss_sex = users.sex.isna().sum()

print("Всего пользователей: ",l)
print("Отсутствует возраст: ", miss_age)
print("Отсутствует пол: ",miss_sex)


Всего пользователей:  142888
Отсутствует возраст:  146
Отсутствует пол:  6262


Как можем видеть, пропусков в таблице пропусков совсем мало. Возраст пусть будет категориальным прзнаком, а не числовым.

### Items

In [11]:
items.sample(10)

,id,title,genres,authors,year
41747,154737,Добро пожаловать в ад. Репортажи с войны в Чечне,Современная русская литература,Валерий Киселев,NaN
17381,103190,Возмездие. Никогда не поздно,Боевики,Михаил Нестеров,2013
32182,196049,Главный секрет успешного имиджмейкера. Persona...,"Рукоделие и ремесла,Биографии и мемуары,Соврем...","Maikl Sosnin,Анна Мурашко",NaN
49864,288960,"Масштабирование, или Как заставить ваш бизнес ...","Личная эффективность,Истории успеха,Управление...",Евгений Ойстачер,NaN
59108,221191,Лес теней,"Зарубежные детективы,Триллеры,Современные дете...",Франк Тилье,2006
42785,24570,В стране Недоделкино и другие сказки,"Книги для детей,Детские приключения,Сказки",Анна Гале,2018
18709,10713,"Маленькая балерина (Антонина Нестеровская, кня...",Исторические любовные романы,Елена Арсеньева,NaN
54359,153973,Домашний Журнал 13-2019,"Прикладная литература,Журнальные издания",NaN,2019
49877,57121,Моя жирная логика. Как выбросить из головы мус...,"Похудение и диеты,Здоровое и правильное питание",Надя Германн,2016
45294,295445,Тайны Звезд 38-2016,Газеты,NaN,2016


Структура этого датафрейма уже несколько сложнее:

1) В графе год может быть задан век, период, перечисление годов.
2) может быть несколько авторов
3) Графа geners задана как перечисления жанров


In [12]:
non_numeric = items[~items.year.apply(lambda x: str(x).isdigit())]

non_numeric

,id,title,genres,authors,year
5,250772,Исповедь,"Стихи и поэзия,Литература 19 века",Михаил Лермонтов,1829-1830
8,307,Путём-дорогою,"Зарубежные детские книги,Сказки,Русская классика",Михаил Салтыков-Щедрин,NaN
9,133138,Соседи,"Зарубежные детские книги,Сказки,Русская классика",Михаил Салтыков-Щедрин,NaN
13,285394,Хитрость,"Зарубежная классика,Классическая проза",Ги де Мопассан,NaN
16,99353,Записки о Галльской войне,"Зарубежная старинная литература,Античная литер...",Гай Юлий Цезарь,58-50 гг. до н. э.
...,...,...,...,...,...
59367,191687,Держи марку! Делай деньги! (сборник),"Зарубежное фэнтези,Юмористическое фэнтези,Клас...",Терри Пратчетт,"2004, 2007"
59414,28845,Земля,Современная русская литература,Михаил Елизаров,2014–2019
59457,86224,"8500 слов, одинаковых на русском и английском ...",Языкознание,Владимир Струг,NaN
59462,170805,Быть писателем. Дизайн обложки книги. Портфолио,"Рукоделие и ремесла,Руководства,Публицистика",Татьяна Олива Моралес,NaN


In [13]:
proc = len(non_numeric)/len(items)

empty = items.year.isna().sum()/len(items)

print("Доля нечсиловых годов: ", round(proc * 100),'%')
print("Доля пустых данных по всей выборке: ", round(empty * 100),'%')
print("Доля пустых данные среди всех нечисловых годов: ", round(empty/proc * 100), '%')

Доля нечсиловых годов:  23 %
Доля пустых данных по всей выборке:  22 %
Доля пустых данные среди всех нечисловых годов:  94 %


Как видим, большинство 'сломанных годов' - это просто пропуски, и лишь 6% от них - диапазоны, перечисления и века.

Тем не менее, доля "плохих" годов слишком высока и, просто задав каждому пропуску среднее, мы потеряем много информации. Предлагаю следующую стратегию:

- Для каждого пропуска - поставить среднее от времени активности автора (на основе других записей)
- Если не получается восстановить время активности автора по другим записям, то можно загуглить (если случаев будет мало), или просто забить и пусть CatBoost сам разбиарается 🙃

Поизучаем авторов.
Во-первых, видим, что в одной графе может быть перечисление авторов.

In [14]:
empty_auth = items.authors.isna().sum()/len(items)
print("Доля пустых данных по всей выборке: ", round(empty_auth * 100),'%')

Доля пустых данных по всей выборке:  12 %


In [15]:
items[items.authors.isna()]

,id,title,genres,authors,year
23,92503,Подземные царства,Сказки,NaN,NaN
29,35488,У страха глаза велики,Сказки,NaN,NaN
56,231627,Царство мрачного Аида,"Мифы / легенды / эпос,Зарубежные детские книги...",NaN,NaN
57,71261,Орфей и Эвридика,"Мифы / легенды / эпос,Зарубежные детские книги...",NaN,NaN
65,277888,Бой на калиновом мосту,Сказки,NaN,NaN
...,...,...,...,...,...
59594,45640,МК Московский Комсомолец 291-2019,"Политология,Книги по экономике,Газеты",NaN,2019
59595,321616,МК Московский Комсомолец 292-2019,"Политология,Книги по экономике,Газеты",NaN,2019
59596,125582,Известия 248-249-2019,"Политология,Общая история,Газеты",NaN,2019
59597,33188,Men's Health 01-2020,Журнальные издания,NaN,2019


Процент пропущенных авторов немаленький. Но, сделать ничего нельзя 😢

Выясним количество уникальных авторов.

In [16]:
uniq_auth = items.authors.dropna().str.split(r'[^\w\s\-]', regex=True).explode()
uniq_auth = uniq_auth.drop_duplicates()
uniq_auth

0        Михаил Салтыков-Щедрин
1                   Антон Чехов
3              Михаил Лермонтов
12              Уильям Теккерей
13               Ги де Мопассан
                  ...          
59479                 Эрика Хан
59535              Андрей Зорин
59566               Клара Гримм
59572               Май Шёвалль
59591      Евлампий Бесподобный
Name: authors, Length: 19432, dtype: object

Авторов не так много как казалось, но для бустинга, такое количество никуда не годится.

А вдруг есть дупликаты - например, Михаил Лермонтов и Михаил Юрьевич Леромонтов? - эта проблема весьма нетривиальная, пока будем считать, что информация об авторе не столь важна.



Информацию об авторах будем использовать для создания фич следующим образом:
- Рейтинг автора на основе оценок
- Количество прочтений (прочтением будем считать прогресс > 10%)

Осталось, пожалуй, самое важное - жанры.

Вновь посчитаем пропуски.

In [17]:
empty_gen = items.genres.isna().sum()/len(items)
print("Доля пустых данных по всей выборке: ", round(empty_gen * 100),'%')

Доля пустых данных по всей выборке:  0 %


Отсутсвуют пропуски для самого важного признака - очень хорошо.

Выделим все уникальные жанры.

In [18]:
uniq_gnres = items.genres.dropna().str.split(',').explode()
uniq_gnres = uniq_gnres.drop_duplicates()
uniq_gnres

0                      Зарубежные детские книги
0                                        Сказки
0                           Зарубежная классика
0                            Литература 19 века
0                              Русская классика
                          ...                  
58413    Школьные учебники по английскому языку
58414                          Биология 9 класс
58415                         ГИА по химии (ОГЭ
58415                             Химия 9 класс
58953               Школьные учебники по музыке
Name: genres, Length: 640, dtype: object

Обрабатываться будут следующим образом:
- Переведём их в векторные эмбеденги
- Кластеризуем на, скажем, 20 - 30 групп методом KNN
- Будем определять, как каким класетром относиться каждая книга.

## Interections




In [19]:
estimates.sample(10)

,user_id,item_id,progress,rating,start_date
879730,17554,117187,100,NaN,2019-02-25
1319214,117099,87779,4,NaN,2019-09-21
362620,111173,84530,7,NaN,2018-06-25
1526309,35418,224735,39,NaN,2019-12-28
632340,61163,33935,15,NaN,2018-10-31
914252,137766,311866,31,NaN,2019-03-13
712734,65707,225121,100,5.0,2018-12-08
53406,152924,89152,1,NaN,2018-01-26
755749,145930,30975,20,NaN,2018-12-28
730329,80752,141019,75,NaN,2018-12-16


Таблица обладает полезной временной информацией. Однако, в полной мере её польза может раскрыться лишь в непрерывном поступлении новых данных. Поэтому, пока ограничимся лишь столбцом rating.

Фичи будут составляться так: raring >= 4 - метка класса 1, иначе 0.

# Обработка данных

### Рейтинг авторов

In [20]:
def extract_authors_fast(item_id, items_cache=None):
    """Быстрая версия extract_authors с кэшированием"""
    if items_cache is None:
        items_cache = {}

    if item_id in items_cache:
        return items_cache[item_id]

    mask = items['id'] == item_id
    if not mask.any():
        items_cache[item_id] = []
        return []

    authors_str = items.loc[mask, 'authors'].iloc[0]
    if pd.isna(authors_str):
        items_cache[item_id] = []
        return []

    result = re.split(r'[^\w\s\-]', authors_str)
    items_cache[item_id] = result
    return result


# Предварительная фильтрация и подготовка данных
valid_estimates = estimates[
    estimates['rating'].notna() &
    (estimates['progress'] >= 10)
].copy()



In [21]:

def get_year(item_id):
  mask = items['id'] == item_id

  if not mask.any():
    return None

  raw_year = items.loc[mask, 'year'].iloc[0]
  if pd.isna(raw_year):
    return None

  years = re.findall(r'\d+', raw_year)

  if len(years) == 0:
    return None

  list_years = list(map(int, years))
  return sum(list_years)/len(list_years)


In [22]:

# Основной оптимизированный код
def calculate_auth_rating_fast():
    # Предварительная фильтрация и группировка
    valid_estimates = estimates[
        estimates['rating'].notna() |
        (estimates['progress'] >= 10)
    ]

    # Группируем сразу все нужные данные
    item_data = valid_estimates.groupby('item_id').agg({
        'rating': list,
    })

    # Предзагружаем годы для всех items
    item_years = {}
    for item_id in item_data.index:
        item_years[item_id] = get_year(item_id)

    # Создаем словарь для авторов
    auth_rating = {}

    for item_id, row in item_data.iterrows():
        authors = extract_authors_fast(item_id)
        if not authors:
            continue


        ratings = [r for r in row['rating'] if pd.notna(r)]
        year = item_years[item_id]
        rating_sum = sum(map(int,ratings))
        rating_count = len(ratings)

        # Вычисляем сумму годов если год есть
        year_sum = year * rating_count if year is not None else 0
        year_count = rating_count if year is not None else 0

        for author in authors:
            if author not in auth_rating:
                auth_rating[author] = [0, 0, 0, 0]  # [count, sum_ratings, sum_years, year_count]

            auth_rating[author][0] += rating_count
            auth_rating[author][1] += rating_sum
            auth_rating[author][2] += year_sum
            auth_rating[author][3] += year_count

    return auth_rating

auth_rating = calculate_auth_rating_fast()
print(auth_rating)

{'Роберт Аллен': [3, 14, 6030.0, 3], 'Тацунари Иота': [6, 30, 12084.0, 6], 'Пенелопа Дуглас': [276, 1242, 556060.0, 276], 'Геннадий Мурзин': [7, 33, 0, 0], 'Екатерина Лебедева': [49, 203, 2018.0, 1], 'Елена Прудникова': [7, 35, 14116.0, 7], 'Сергей Кремлев': [26, 118, 46321.0, 23], 'Игорь Власов': [18, 71, 36276.0, 18], 'Корнелия Функе': [41, 186, 82242.0, 41], 'Клэр Макинтош': [6, 26, 12098.0, 6], 'Джеймс Дэшнер': [19, 82, 38186.0, 19], 'Владимир Набоков': [102, 452, 156191.0, 80], 'Роберт Льюис Стивенсон': [94, 431, 165758.0, 87], 'Дарья Сойфер': [23, 105, 46420.0, 23], 'Любовь Марченко': [5, 24, 0, 0], 'Анастасия Красичкова': [6, 27, 12075.0, 6], 'Николя Рыженков': [1, 5, 0, 0], 'Лион Фейхтвангер': [5, 19, 9911.0, 5], 'Роман Антропов': [14, 67, 26642.0, 14], 'Равиль Бикбаев': [4, 20, 8072.0, 4], 'Светлана Кудрина': [1, 5, 2010.0, 1], 'Александр Пушкин': [644, 3016, 1055067.0, 575], 'Артелина Грудина': [3, 14, 6051.0, 3], 'Сергей Есенин': [164, 755, 103489.0, 53], 'Макс Фрай': [811, 

In [23]:
print(auth_rating)

{'Роберт Аллен': [3, 14, 6030.0, 3], 'Тацунари Иота': [6, 30, 12084.0, 6], 'Пенелопа Дуглас': [276, 1242, 556060.0, 276], 'Геннадий Мурзин': [7, 33, 0, 0], 'Екатерина Лебедева': [49, 203, 2018.0, 1], 'Елена Прудникова': [7, 35, 14116.0, 7], 'Сергей Кремлев': [26, 118, 46321.0, 23], 'Игорь Власов': [18, 71, 36276.0, 18], 'Корнелия Функе': [41, 186, 82242.0, 41], 'Клэр Макинтош': [6, 26, 12098.0, 6], 'Джеймс Дэшнер': [19, 82, 38186.0, 19], 'Владимир Набоков': [102, 452, 156191.0, 80], 'Роберт Льюис Стивенсон': [94, 431, 165758.0, 87], 'Дарья Сойфер': [23, 105, 46420.0, 23], 'Любовь Марченко': [5, 24, 0, 0], 'Анастасия Красичкова': [6, 27, 12075.0, 6], 'Николя Рыженков': [1, 5, 0, 0], 'Лион Фейхтвангер': [5, 19, 9911.0, 5], 'Роман Антропов': [14, 67, 26642.0, 14], 'Равиль Бикбаев': [4, 20, 8072.0, 4], 'Светлана Кудрина': [1, 5, 2010.0, 1], 'Александр Пушкин': [644, 3016, 1055067.0, 575], 'Артелина Грудина': [3, 14, 6051.0, 3], 'Сергей Есенин': [164, 755, 103489.0, 53], 'Макс Фрай': [811, 

In [24]:
fixed_auth_rating = {}
for author, values in auth_rating.items():
    fixed_auth_rating[author] = values.copy()  # или list(values)


for auth in fixed_auth_rating.keys():


  if fixed_auth_rating[auth][0] != 0:
    fixed_auth_rating[auth][1] = fixed_auth_rating[auth][1] /fixed_auth_rating[auth][0]
  else:
    fixed_auth_rating[auth][1] = None


  if fixed_auth_rating[auth][3] != 0:
    fixed_auth_rating[auth][2] = fixed_auth_rating[auth][2] /fixed_auth_rating[auth][3]
  else:
    fixed_auth_rating[auth][2] = None
  # auth_rating[auth][2] = [auth_rating[auth][2]/auth_rating[auth][3]]

print(fixed_auth_rating)

{'Роберт Аллен': [3, 4.666666666666667, 2010.0, 3], 'Тацунари Иота': [6, 5.0, 2014.0, 6], 'Пенелопа Дуглас': [276, 4.5, 2014.7101449275362, 276], 'Геннадий Мурзин': [7, 4.714285714285714, None, 0], 'Екатерина Лебедева': [49, 4.142857142857143, 2018.0, 1], 'Елена Прудникова': [7, 5.0, 2016.5714285714287, 7], 'Сергей Кремлев': [26, 4.538461538461538, 2013.9565217391305, 23], 'Игорь Власов': [18, 3.9444444444444446, 2015.3333333333333, 18], 'Корнелия Функе': [41, 4.536585365853658, 2005.9024390243903, 41], 'Клэр Макинтош': [6, 4.333333333333333, 2016.3333333333333, 6], 'Джеймс Дэшнер': [19, 4.315789473684211, 2009.7894736842106, 19], 'Владимир Набоков': [102, 4.431372549019608, 1952.3875, 80], 'Роберт Льюис Стивенсон': [94, 4.585106382978723, 1905.2643678160919, 87], 'Дарья Сойфер': [23, 4.565217391304348, 2018.2608695652175, 23], 'Любовь Марченко': [5, 4.8, None, 0], 'Анастасия Красичкова': [6, 4.5, 2012.5, 6], 'Николя Рыженков': [1, 5.0, None, 0], 'Лион Фейхтвангер': [5, 3.8, 1982.2, 5]

Заполняем пропуски годов, если возможно.


In [25]:
def fill_na_in_year(row):
    """Функция для применения через apply к строкам DataFrame"""
    item_id = row['id']  # или row.name если индекс это item_id
    auths = extract_authors_fast(item_id)

    if not auths:
        return None

    max_year = 0
    for author in auths:
        if author in fixed_auth_rating:
            # fixed_auth_rating[author][2] содержит сумму годов
            # fixed_auth_rating[author][3] содержит количество
            if fixed_auth_rating[author][3] > 0:  # проверяем, что есть годы
                avg_year = fixed_auth_rating[author][2] / fixed_auth_rating[author][3]
                max_year = max(max_year, avg_year)

    return max_year if max_year > 0 else None



In [26]:
na_mask = items['year'].isna()

items.loc[na_mask, 'year'] = items[na_mask].apply(fill_na_in_year, axis=1)

In [27]:
len(items[items.year.isna()])

7300

Количество пропусков по годам уменьшилось значительно!

In [28]:
items['year'] = items['year'].fillna(1984)

# Затем исправляем нечисловые значения через loc
mask = ~items['year'].apply(lambda x: str(x).isdigit())
items.loc[mask, 'year'] = 1984

In [29]:
items[items.year.isna()]

,id,title,genres,authors,year


### Жанры

---



Представим каждый жанр как вектор.

In [30]:
from sentence_transformers import SentenceTransformer

# Используем модель, обученную на русских текстах
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

genres = uniq_gnres.tolist()

# Кодируем напрямую названия жанров
genre_vectors = model.encode(genres)


ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
from sklearn.cluster import KMeans

# Определяем количество кластеров
n_clusters = 50 # можно подобрать экспериментально

# Выполняем кластеризацию
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(genre_vectors)

# Создаем DataFrame для наглядности
import pandas as pd
result_df = pd.DataFrame({
    'Жанр': genres,
    'Кластер': clusters
})

print("РАЗБИЕНИЕ НА КЛАСТЕРЫ:")
print("=" * 30)
for cluster_num in range(n_clusters):
    cluster_genres = result_df[result_df['Кластер'] == cluster_num]['Жанр'].tolist()
    print(f"Кластер {cluster_num}: {', '.join(cluster_genres)}")

РАЗБИЕНИЕ НА КЛАСТЕРЫ:
Кластер 0: Зарубежное фэнтези, Детективное фэнтези, Боевое фэнтези, Анекдоты, Боевики, Кулинария, Триллеры, Сделай сам, Руководства, Попаданцы, Городское фэнтези, Трейдинг, Программы, Йога, Киберпанк, Автомобили и ПДД, Стимпанк, Ужасы, Недвижимость, Фэншуй / фэн-шуй, Рыбалка, Автотранспорт, Этнография, Вестерны, Цирк, Английский язык, НЛП, Легкая проза, Интеллектуальные игры, Музыка, Ботаника, Эротическое фэнтези, Охота, Парикмахерское дело, Брендинг, Коучинг, Зарубежные боевики, Фольклор, Советы от гуру, Разговорники, Интерьеры, Сборники, Задачники, Риски, Лояльность, Ветеринария, Механика, Эстетика, Тимбилдинг, Воздушный транспорт, Фанфик, Железнодорожный транспорт, Буквари, Космический транспорт, Машиностроение, Безалкогольные напитки, Cемья,  ГВЭ),  образцы, Акафисты, Альманахи, Дистрибуция, Водный транспорт, Мерчендайзинг
Кластер 1: Банковское дело, Финансовое право, Работа с долгами
Кластер 2: Зарубежные детские книги, Список школьной литературы 5-6 класс, 

Встречаются иногда забавные: Русское фэнтази -> Русский язык. Но это поправимо, если написать описание для каждого жанра. Но мне лень этим заниматься. Быть может, исправлю в дальнейшем, так как в остальном качество меня пока устраиват.

Ладно, распределим каждую книгу по кластерам, причём одна книга может находиться в нескольких кластерах.

In [ ]:
def extract_gnres(item_id):
  mask = items['id'] == item_id

  if not mask.any():
    return None

  raw_genres = items.loc[mask, 'genres'].iloc[0]

  if pd.isna(raw_genres):
    return None

  genres = re.split(r'[^\w\s\-]', raw_genres)


  return genres

genre_to_cluster_dict = dict(zip(result_df['Жанр'], result_df['Кластер']))

def set_clasters(row):

    row_id = row['id']

    genres = extract_gnres(row_id)

    if not genres:
        return None

    # Используем множества для быстрого устранения дубликатов
    clusters = set()
    for genre in genres:
        cluster_id = genre_to_cluster_dict.get(genre)
        if cluster_id is not None:
            clusters.add(cluster_id)

    return  pd.Series([row_id, list(clusters)])



In [ ]:
set_clasters(items[items.id == 1].iloc[0])

,0
0,1
1,"[9, 11, 12]"


In [ ]:
items_with_clasters = items.apply(set_clasters,axis=1)

In [ ]:
items_with_clasters

,0,1
0,128115.0,"[2, 4, 36, 5, 16]"
1,210979.0,"[16, 36, 4]"
2,95632.0,"[16, 36, 4]"
3,247906.0,[36]
4,294280.0,"[16, 36, 4]"
...,...,...
59594,45640.0,"[9, 20, 30]"
59595,321616.0,"[9, 20, 30]"
59596,125582.0,"[9, 5, 30]"
59597,33188.0,[9]


In [ ]:
items_with_clasters.columns = ['id_item', 'cluster_codes']
items_with_clasters

,id_item,cluster_codes
0,128115.0,"[2, 4, 36, 5, 16]"
1,210979.0,"[16, 36, 4]"
2,95632.0,"[16, 36, 4]"
3,247906.0,[36]
4,294280.0,"[16, 36, 4]"
...,...,...
59594,45640.0,"[9, 20, 30]"
59595,321616.0,"[9, 20, 30]"
59596,125582.0,"[9, 5, 30]"
59597,33188.0,[9]


### Склеивание


In [ ]:
#почистим оценки

clear_estimates = estimates.dropna(subset=['rating'])

In [ ]:
merged_with_users = clear_estimates.merge(
    users[['user_id', 'age', 'sex']],
    on='user_id',
    how='inner')  # ← ТОЛЬКО СОВПАДАЮЩИЕ!)

merged_with_users

,user_id,item_id,progress,rating,start_date,age,sex
0,46791,83486,23,5.0,2018-01-01,18_24,1.0
1,79313,188770,88,5.0,2018-01-01,55_64,0.0
2,42797,315927,69,5.0,2018-01-01,25_34,NaN
3,18548,153547,42,5.0,2018-01-01,18_24,0.0
4,91177,52112,28,2.0,2018-01-01,35_44,0.0
...,...,...,...,...,...,...,...
262620,80070,157666,100,3.0,2019-12-31,45_54,0.0
262621,88707,193170,100,5.0,2019-12-31,25_34,1.0
262622,50059,320791,35,5.0,2019-12-31,45_54,0.0
262623,124840,227664,94,5.0,2019-12-31,35_44,0.0


In [ ]:
merged_with_items = merged_with_users.merge(
    items[['id', 'genres', 'authors', 'year']],
    left_on='item_id',
    right_on='id',
    how='inner'  # ← ТОЛЬКО СОВПАДАЮЩИЕ!
)

merged_with_items

,user_id,item_id,progress,rating,start_date,age,sex,id,genres,authors,year
0,46791,83486,23,5.0,2018-01-01,18_24,1.0,83486,Здоровье,Ванесса Томпсон,2003
1,79313,188770,88,5.0,2018-01-01,55_64,0.0,188770,"Стихи и поэзия,Русская классика",Игорь Северянин,1984
2,42797,315927,69,5.0,2018-01-01,25_34,NaN,315927,"Боевая фантастика,Космическая фантастика",Андрей Ворон (Mr. Incognito2),2019
3,18548,153547,42,5.0,2018-01-01,18_24,0.0,153547,"Зарубежные любовные романы,Короткие любовные р...",Карла Кэссиди,2018
4,91177,52112,28,2.0,2018-01-01,35_44,0.0,52112,Современные детективы,Галина Романова,2009
...,...,...,...,...,...,...,...,...,...,...,...
262620,80070,157666,100,3.0,2019-12-31,45_54,0.0,157666,"Попаданцы,Любовное фэнтези",Алиса Ардова,2016
262621,88707,193170,100,5.0,2019-12-31,25_34,1.0,193170,"Исторические детективы,Шпионские детективы",Борис Акунин,2011
262622,50059,320791,35,5.0,2019-12-31,45_54,0.0,320791,Боевики,Александр Золотько,1984
262623,124840,227664,94,5.0,2019-12-31,35_44,0.0,227664,"Парапсихология,Эзотерика / оккультизм,Биографи...",Эбен Александер,2012


In [ ]:
merged_with_items = merged_with_items.merge(
    items_with_clasters[['id_item','cluster_codes']],
    left_on='item_id',
    right_on='id_item',
    how='inner'  # ← ТОЛЬКО СОВПАДАЮЩИЕ!

)

In [ ]:
merged_with_items

,user_id,item_id,progress,rating,start_date,age,sex,id,genres,authors,year,id_item,cluster_codes
0,46791,83486,23,5.0,2018-01-01,18_24,1.0,83486,Здоровье,Ванесса Томпсон,2003,83486.0,[10]
1,79313,188770,88,5.0,2018-01-01,55_64,0.0,188770,"Стихи и поэзия,Русская классика",Игорь Северянин,1984,188770.0,"[36, 4]"
2,42797,315927,69,5.0,2018-01-01,25_34,NaN,315927,"Боевая фантастика,Космическая фантастика",Андрей Ворон (Mr. Incognito2),2019,315927.0,[7]
3,18548,153547,42,5.0,2018-01-01,18_24,0.0,153547,"Зарубежные любовные романы,Короткие любовные р...",Карла Кэссиди,2018,153547.0,[29]
4,91177,52112,28,2.0,2018-01-01,35_44,0.0,52112,Современные детективы,Галина Романова,2009,52112.0,[18]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
262556,80070,157666,100,3.0,2019-12-31,45_54,0.0,157666,"Попаданцы,Любовное фэнтези",Алиса Ардова,2016,157666.0,"[0, 48]"
262557,88707,193170,100,5.0,2019-12-31,25_34,1.0,193170,"Исторические детективы,Шпионские детективы",Борис Акунин,2011,193170.0,[18]
262558,50059,320791,35,5.0,2019-12-31,45_54,0.0,320791,Боевики,Александр Золотько,1984,320791.0,[0]
262559,124840,227664,94,5.0,2019-12-31,35_44,0.0,227664,"Парапсихология,Эзотерика / оккультизм,Биографи...",Эбен Александер,2012,227664.0,"[3, 15]"


добавим ещё статистик для авторов.

In [ ]:
# # fixed_auth_rating

# def add_author_stats(row):
#   auths = extract_authors_fast(row['item_id'])

#   if auths == []:
#     return None

#   auth = auths[0]

#   auth_data = fixed_auth_rating[auth]

#   return pd.Series({
#       'author_rating' : auth_data[1],
#       'author_popularity':auth_data[0]
#   })


all_item_ids = merged_with_items['item_id'].unique()

# Предварительно извлечем всех авторов
item_to_author = {}
for item_id in all_item_ids:
    auths = extract_authors_fast(item_id)
    item_to_author[item_id] = auths[0] if auths else None

# Сопоставляем авторов с рейтингами
author_ratings = []
author_popularities = []

for item_id in merged_with_items['item_id']:
    author = item_to_author.get(item_id)
    if author and author in fixed_auth_rating:
        auth_data = fixed_auth_rating[author]
        author_ratings.append(auth_data[1])
        author_popularities.append(auth_data[0])
    else:
        author_ratings.append(None)
        author_popularities.append(None)

# Добавляем столбцы сразу
merged_with_items['author_rating'] = author_ratings
merged_with_items['author_popularity'] = author_popularities



In [ ]:
merged_with_items

,user_id,item_id,progress,rating,start_date,age,sex,id,genres,authors,year,id_item,cluster_codes,author_rating,author_popularity
0,46791,83486,23,5.0,2018-01-01,18_24,1.0,83486,Здоровье,Ванесса Томпсон,2003,83486.0,[10],4.210526,19.0
1,79313,188770,88,5.0,2018-01-01,55_64,0.0,188770,"Стихи и поэзия,Русская классика",Игорь Северянин,319.958333,188770.0,"[36, 4]",4.923077,13.0
2,42797,315927,69,5.0,2018-01-01,25_34,NaN,315927,"Боевая фантастика,Космическая фантастика",Андрей Ворон (Mr. Incognito2),2019,315927.0,[7],5.000000,3.0
3,18548,153547,42,5.0,2018-01-01,18_24,0.0,153547,"Зарубежные любовные романы,Короткие любовные р...",Карла Кэссиди,2018,153547.0,[29],4.777778,9.0
4,91177,52112,28,2.0,2018-01-01,35_44,0.0,52112,Современные детективы,Галина Романова,2009,52112.0,[18],4.631687,486.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262556,80070,157666,100,3.0,2019-12-31,45_54,0.0,157666,"Попаданцы,Любовное фэнтези",Алиса Ардова,2016,157666.0,"[0, 48]",4.526316,38.0
262557,88707,193170,100,5.0,2019-12-31,25_34,1.0,193170,"Исторические детективы,Шпионские детективы",Борис Акунин,2011,193170.0,[18],4.650450,2775.0
262558,50059,320791,35,5.0,2019-12-31,45_54,0.0,320791,Боевики,Александр Золотько,25.15,320791.0,[0],4.531250,96.0
262559,124840,227664,94,5.0,2019-12-31,35_44,0.0,227664,"Парапсихология,Эзотерика / оккультизм,Биографи...",Эбен Александер,2012,227664.0,"[3, 15]",4.583333,12.0


# Обучение модели

In [ ]:
necessery_colums = ['age','sex','year','cluster_codes','author_rating', 'author_popularity','rating']

XY = merged_with_items[necessery_colums]

XY

,age,sex,year,cluster_codes,author_rating,author_popularity,rating
0,18_24,1.0,2003,[10],4.210526,19.0,5.0
1,55_64,0.0,1984,"[36, 4]",4.923077,13.0,5.0
2,25_34,NaN,2019,[7],5.000000,3.0,5.0
3,18_24,0.0,2018,[29],4.777778,9.0,5.0
4,35_44,0.0,2009,[18],4.631687,486.0,2.0
...,...,...,...,...,...,...,...
262556,45_54,0.0,2016,"[0, 48]",4.526316,38.0,3.0
262557,25_34,1.0,2011,[18],4.650450,2775.0,5.0
262558,45_54,0.0,1984,[0],4.531250,96.0,5.0
262559,35_44,0.0,2012,"[3, 15]",4.583333,12.0,5.0


In [31]:
df_processed = XY.copy()

all_clusters = set()
for clusters in df_processed['cluster_codes']:
    all_clusters.update(clusters)

# Создаем колонки и преобразуем в категориальный тип
for cluster in all_clusters:
    df_processed[f'cluster_{cluster}'] = df_processed['cluster_codes'].apply(lambda x: cluster in x)
    df_processed[f'cluster_{cluster}'] = df_processed[f'cluster_{cluster}'].astype('category')

df_processed = df_processed.drop(['cluster_codes'],axis=1)

# Указываем ВСЕ кластеры как категориальные признаки
categorical_features = (['age', 'sex'] +
                       [f'cluster_{cluster}' for cluster in all_clusters])

NameError: name 'XY' is not defined

In [396]:
#Перейдём к бинарной классификации

df_processed['rating'] = df_processed.apply(lambda x :  1 if int(x['rating']) > 3 else 0, axis=1)

In [397]:
#починим Nan


df_processed['sex'] = df_processed['sex'].astype('string')
df_processed['sex'] = df_processed['sex'].fillna('NaN')

df_processed['age'] = df_processed['age'].fillna('35_44')

In [398]:
df_processed.sample(10)

,age,sex,year,author_rating,author_popularity,rating,cluster_0,cluster_1,cluster_2,cluster_3,...,cluster_40,cluster_41,cluster_42,cluster_43,cluster_44,cluster_45,cluster_46,cluster_47,cluster_48,cluster_49
249289,55_64,NaN,2019,NaN,NaN,1,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
223025,18_24,1.0,1997,4.613123,2789.0,1,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
241539,55_64,NaN,2014,3.500000,2.0,1,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15658,25_34,0.0,1984,4.657143,35.0,1,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
260456,18_24,0.0,2017,4.641026,234.0,1,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
206387,25_34,0.0,2010,4.731544,298.0,1,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
231311,45_54,1.0,2017,4.738318,214.0,1,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
226357,45_54,0.0,2015,4.660714,168.0,1,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
231270,45_54,1.0,2013,4.600000,10.0,1,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
52052,55_64,0.0,2018,NaN,NaN,1,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [399]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(df_processed.drop(['rating'],axis = 1),df_processed['rating'],train_size = 0.7,random_state=42)


In [400]:
print(Y_train.value_counts())

rating
1    164248
0     19544
Name: count, dtype: int64


Наблюдается сильный дизбаланс классов. Поправим.

In [401]:
from sklearn.utils import compute_class_weight

classes = np.unique(Y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=Y_train)
class_weights = dict(zip(classes, weights))

Теперь, момент истины!

In [199]:
!pip install catboost

In [428]:
from catboost import CatBoostClassifier

categorial = categorical_features
# categorial = ['age','sex','cluster_codes']
# + [f'cluster_{i}' for i in range(max_cluster_length)]

model = CatBoostClassifier(
    iterations = 2000,
    learning_rate=0.5,
    cat_features=categorial,
    loss_function='Logloss',
    # eval_metric='Accuracy',
    # random_seed=42,
    verbose=100,
    class_weights=class_weights
)

In [429]:
model.fit(X_train,Y_train)

0:	learn: 0.6550101	total: 210ms	remaining: 6m 59s
100:	learn: 0.5982757	total: 19.2s	remaining: 6m 1s
200:	learn: 0.5807644	total: 39.8s	remaining: 5m 56s
300:	learn: 0.5667254	total: 1m	remaining: 5m 39s
400:	learn: 0.5544858	total: 1m 21s	remaining: 5m 24s
500:	learn: 0.5427252	total: 1m 43s	remaining: 5m 10s
600:	learn: 0.5325976	total: 2m 3s	remaining: 4m 47s
700:	learn: 0.5236298	total: 2m 25s	remaining: 4m 29s
800:	learn: 0.5151279	total: 2m 46s	remaining: 4m 8s
900:	learn: 0.5063355	total: 3m 6s	remaining: 3m 48s
1000:	learn: 0.4989771	total: 3m 28s	remaining: 3m 27s
1100:	learn: 0.4920000	total: 3m 48s	remaining: 3m 6s
1200:	learn: 0.4849056	total: 4m 9s	remaining: 2m 45s
1300:	learn: 0.4787803	total: 4m 29s	remaining: 2m 24s
1400:	learn: 0.4724574	total: 4m 50s	remaining: 2m 4s
1500:	learn: 0.4671316	total: 5m 10s	remaining: 1m 43s
1600:	learn: 0.4614264	total: 5m 30s	remaining: 1m 22s
1700:	learn: 0.4558316	total: 5m 51s	remaining: 1m 1s
1800:	learn: 0.4503568	total: 6m 13s	

In [432]:
model.score(X_test,Y_test)

np.float64(0.6786045271617007)

In [445]:
from sklearn.metrics import precision_score, classification_report

# Получаем предсказания
y_pred = model.predict(X_test)


print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.16      0.50      0.25      8317
           1       0.92      0.70      0.80     70452

    accuracy                           0.68     78769
   macro avg       0.54      0.60      0.52     78769
weighted avg       0.84      0.68      0.74     78769



Так как эта модель будет применяться на наиболее вероятных кандидатах в рекомендации, нас такая статистика пока устроит.

In [446]:
#скачиваем модель

model.save_model('rec_cat.ctb')
